In [1]:
import sklearn.preprocessing
import tensorboard
import tensorflow as tf
import numpy as np
import keras.backend
import datetime

from packaging import version
from linkmethods import get_target, get_backlink_views
from keras.layers import *
from sklearn.model_selection import train_test_split

In [2]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.11.0


In [ ]:
article = "Data_Science"

In [ ]:
#defining a cutoff, a lower cutoff provides a lower difference across all the input data. thus probably more accuracy

In [ ]:
cutoff = -512

In [ ]:
df_main = get_target(article)
bl_views = get_backlink_views(article, get_target(article))

In [ ]:
#getting the Data and target data for the article, then printing them both to get a look at them.

In [ ]:
data = np.array(bl_views)[:,cutoff:-1]
print(data)
data.shape

In [ ]:
target = np.array(df_main.views, dtype=float)[cutoff:-1]
target = np.array([np.copy(target) for _ in range((data.shape[0]))])
print(target)
target.shape

In [ ]:
#the loss function

In [ ]:
def smape(tftarget, tfforecast):
    denominator = tf.reduce_sum(tf.abs(tftarget)+tf.abs(tfforecast))
    if tf.reduce_sum(tftarget)+tf.reduce_sum(tfforecast) == 0:
        return tf.cast(1.0,tf.float64)
    result = (2/len(tftarget)) * tf.cast(tf.reduce_sum(tf.abs(tftarget - tfforecast))/denominator,tf.float64)

    return result

In [ ]:
#model has the size of the data you input into.

In [ ]:
# Define the model.
model = keras.Sequential()

model.add(Input(shape=-cutoff-1)) # Input tensor
model.add(Dense(units=-cutoff-1)) # hidden layer 1,
model.add(Activation(activation="sigmoid"))

model.compile(
    optimizer='Adam',
    loss=smape,
    metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.summary()

In [ ]:
#plotting the model

In [ ]:
keras.utils.plot_model(model, "my_first_model.png")

In [ ]:
#normalizing the data to even out the values

In [ ]:
predata = sklearn.preprocessing.normalize(data,norm="l1")
pretarget = sklearn.preprocessing.normalize(target,norm="l1")

In [ ]:
# Separate the test data
x, x_test, y, y_test = train_test_split(predata, pretarget, test_size=0.15, shuffle=True)

# Split the remaining data to train and validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, shuffle=True)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
#running this cell will make the NN run with the data that it has done progess on again!
#define the model again for a clean start.

In [ ]:
# Train the model.
early_stop = keras.callbacks.EarlyStopping(monitor="accuracy",patience=150, restore_best_weights=True)
history = model.fit(
    x_train,
    y_train,
    batch_size=4,
    epochs=150,
    validation_data=(x_val, y_val),
    callbacks=early_stop)

In [ ]:
from matplotlib import pyplot as plt
def display_learning_curves(history):
    fig, axs = plt.subplots(2, 2, figsize=(19, 11))

    axs[0,0].plot(history.history["loss"])
    axs[0,0].legend(["train", "test"], loc="upper right")
    axs[0,0].set_xlabel("Epochs")
    axs[0,0].set_ylabel("Loss")

    axs[0,1].plot(history.history["accuracy"])
    axs[0,1].legend(["train", "test"], loc="upper right")
    axs[0,1].set_xlabel("Epochs")
    axs[0,1].set_ylabel("Accuracy")

    axs[1,0].plot(history.history["val_loss"])
    axs[1,0].legend(["train", "test"], loc="upper right")
    axs[1,0].set_xlabel("Epochs")
    axs[1,0].set_ylabel("val_loss")

    axs[1,1].plot(history.history["val_accuracy"])
    axs[1,1].legend(["train", "test"], loc="upper right")
    axs[1,1].set_xlabel("Epochs")
    axs[1,1].set_ylabel("val_Accuracy")
    plt.show()

In [ ]:
#plotting the data

In [ ]:
display_learning_curves(history)

In [ ]:
#getting the weights used for this run

In [ ]:
weights = model.get_weights()
print(sum(history.history['loss']) / len(history.history['loss']))
weights